In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# # get data files
# !wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

# !unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
df_ratings

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0
...,...,...,...
1149775,276704,1563526298,9.0
1149776,276706,0679447156,0.0
1149777,276709,0515107662,10.0
1149778,276721,0590442449,10.0


In [22]:
# add your code here - consider creating a new cell for each section of code

df_rating_user_count = df_ratings[['user', 'rating']].groupby('user').count().reset_index().rename(columns={'rating': 'rating_count'})
df_rating_user_count = df_rating_user_count[df_rating_user_count['rating_count'] >= 200]
user_to_keep = df_rating_user_count['user'].to_list()


df_rating_isbn_count = df_ratings.groupby('isbn')['rating'].count().reset_index().rename(columns={'rating': 'rating_count'})
df_rating_isbn_count = df_rating_isbn_count[df_rating_isbn_count['rating_count'] >= 100]
isbn_to_keep = df_rating_isbn_count['isbn'].to_list()



df_ratings_filtered = df_ratings.copy()
df_ratings_filtered = df_ratings_filtered[df_ratings_filtered['user'].isin(user_to_keep)]
df_ratings_filtered = df_ratings_filtered[df_ratings_filtered['isbn'].isin(isbn_to_keep)]
df_ratings_filtered

,user,isbn,rating
1456,277427,002542730X,10.0
1469,277427,0060930535,0.0
1471,277427,0060934417,0.0
1474,277427,0061009059,9.0
1484,277427,0140067477,0.0
...,...,...,...
1147304,275970,0804111359,0.0
1147436,275970,140003065X,0.0
1147439,275970,1400031346,0.0
1147440,275970,1400031354,0.0


In [23]:
len(isbn_to_keep), len(user_to_keep)

(731, 905)

In [26]:
df_books

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber
...,...,...,...
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger
271375,0525447644,From One to One Hundred,Teri Sloat
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker
271377,0192126040,Republic (World's Classics),Plato


In [27]:
df_books_filtered = df_books[df_books['isbn'].isin(isbn_to_keep)]
df_books_filtered['isbn'].nunique()

727

In [28]:
df_books_filtered[df_books_filtered['title'].isin(["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True'])]


,isbn,title,author
45,0671888587,I'll Be Seeing You,Mary Higgins Clark
252,0316782505,The Weight of Water,Anita Shreve
3022,0060987561,I Know This Much Is True,Wally Lamb
3703,0345447840,The Surgeon,TESS GERRITSEN


In [ ]:
from tqdm import tqdm
df_book_vectors = df_books_filtered[['isbn', 'title', 'author']].copy()

df_ratings_filtered = df_ratings_filtered[df_ratings_filtered['isbn'].isin(df_books_filtered['isbn'])]

# Create a pivot table
pivot_table = df_ratings_filtered.pivot(index='isbn', columns='user', values='rating')
pivot_table = pivot_table.fillna(0)
pivot_table = pivot_table.reindex(df_books_filtered['isbn'], fill_value=0)

df_book_vectors['rating_vectors'] = list(pivot_table.to_numpy())
df_book_vectors


,isbn,title,author,rating_vectors
18,0440234743,The Testament,John Grisham,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, ..."
26,0971880107,Wild Animus,Rich Shapero,"[0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
27,0345402871,Airframe,Michael Crichton,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
28,0345417623,Timeline,MICHAEL CRICHTON,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...
28072,0425178765,Easy Prey,John Sandford,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
29215,0449223604,M Is for Malice,Sue Grafton,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
30535,0345444884,The Talisman,STEPHEN KING,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
30775,0060008032,Angels,Marian Keyes,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [68]:
# Convert the pivot table to a csr_matrix
rating_vectors = csr_matrix(pivot_table.values)

# Fit the Nearest Neighbors model
model = NearestNeighbors(metric='cosine')
model.fit(rating_vectors)
print(rating_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 12548 stored elements and shape (727, 888)>
  Coords	Values
  (0, 15)	9.0
  (0, 25)	9.0
  (0, 32)	8.0
  (0, 84)	6.0
  (0, 88)	10.0
  (0, 217)	7.0
  (0, 294)	7.0
  (0, 309)	5.0
  (0, 315)	9.0
  (0, 369)	10.0
  (0, 373)	8.0
  (0, 671)	9.0
  (0, 675)	9.0
  (0, 715)	7.0
  (0, 728)	7.0
  (0, 729)	9.0
  (0, 743)	10.0
  (0, 761)	5.0
  (0, 765)	8.0
  (0, 766)	9.0
  (0, 830)	8.0
  (0, 877)	9.0
  (1, 7)	5.0
  (1, 25)	8.0
  (1, 55)	9.0
  :	:
  (724, 329)	10.0
  (724, 331)	10.0
  (724, 368)	9.0
  (724, 376)	10.0
  (724, 529)	10.0
  (724, 609)	10.0
  (724, 610)	8.0
  (724, 637)	8.0
  (724, 701)	10.0
  (724, 736)	8.0
  (724, 846)	7.0
  (724, 879)	10.0
  (725, 25)	8.0
  (725, 398)	8.0
  (725, 408)	7.0
  (725, 511)	8.0
  (725, 636)	7.0
  (725, 805)	7.0
  (725, 851)	6.0
  (725, 854)	8.0
  (725, 876)	8.0
  (726, 25)	5.0
  (726, 80)	7.0
  (726, 312)	10.0
  (726, 627)	8.0


In [69]:
# function to return recommended books - this will be tested
def get_recommends(title = ""):
    try:
        book = df_books_filtered.loc[df_books_filtered["title"] == title]
    except KeyError as e:
        print('Book', e, 'does not exist')
        return

    b = pivot_table.loc[pivot_table.index.isin(book["isbn"])]
    lookup = [x for x in b.values]
    print(len(lookup[0]))
    distance, indice = model.kneighbors(lookup, n_neighbors=6)

    distance =  distance[0][1:]
    indice = indice[0][1:]

    titles = [
        df_books.loc[df_books['isbn'] == pivot_table.iloc[i].name]["title"].values[0]\
        for i in indice
    ]

    recommended = [list(z) for z in zip(titles, distance)][::-1]
    return [title, recommended]

get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")

888


['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['Catch 22', np.float32(0.7939835)],
  ['The Witching Hour (Lives of the Mayfair Witches)', np.float32(0.74486566)],
  ['Interview with the Vampire', np.float32(0.73450685)],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))',
   np.float32(0.53763384)],
  ['The Vampire Lestat (Vampire Chronicles, Book II)',
   np.float32(0.51784116)]]]

In [82]:
# function to return recommended books - this will be tested
def get_recommends(title = ""):
    rating_vector = np.array(df_book_vectors[df_book_vectors['title'] == title]['rating_vectors'].values[0]).reshape(1, -1)
    
    distances, indices = model.kneighbors(rating_vector, n_neighbors=6)
    # Get books based on indices
    recommended_book_titles = df_book_vectors.iloc[indices[0]]['title'].to_list()
    recommended_book_list = [[x,y] for x,y in zip(recommended_book_titles, distances[0])]
    recommended_book_list = [a for a in recommended_book_list if a[0] != title]
    recommended_book_list.reverse()
    return [title, recommended_book_list]
get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['Catch 22', np.float32(0.7939835)],
  ['The Witching Hour (Lives of the Mayfair Witches)', np.float32(0.74486566)],
  ['Interview with the Vampire', np.float32(0.73450685)],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))',
   np.float32(0.53763384)],
  ['The Vampire Lestat (Vampire Chronicles, Book II)',
   np.float32(0.51784116)]]]

In [83]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  print(recommends)
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", np.float32(0.8016211)], ['The Weight of Water', np.float32(0.77085835)], ['The Surgeon', np.float32(0.7699411)], ['I Know This Much Is True', np.float32(0.7677075)], ['The Lovely Bones: A Novel', np.float32(0.7234864)]]]
["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", np.float32(0.8016211)], ['The Weight of Water', np.float32(0.77085835)], ['The Surgeon', np.float32(0.7699411)], ['I Know This Much Is True', np.float32(0.7677075)], ['The Lovely Bones: A Novel', np.float32(0.7234864)]]]
You passed the challenge! 🎉🎉🎉🎉🎉
